In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/Project'

ModuleNotFoundError: No module named 'google'

In [19]:
TARGET_SR = 22000
N_FFT = 1024,
HOP_LENGHT = 512,
N_MELS = 64

In [2]:
import os
import torch
import torchaudio
import pandas as pd

In [6]:
root = './'
os.listdir(root + '/data')

['hombre', 'Instrumental', 'Mujer']

In [7]:
# M/F/I     indica si el artista es Male, Female, Instrumental
# artist    indica el nombre del artista o banda
# album
# song      indica el nombre de la cncion
# localPath indica la ruta de la carpeta donde esta contenida la cancion
# fileTrak  indica el nombre del archivo que contiene la pista de audio
# fileMel  indica el nombre del espectrograma de mel
# sample/full indica si el espectrograma es calculado respecto a toda la cancion o respecto a un sample de 10 seg
df = pd.DataFrame(columns=['M/F/I', 'artist', 'album', 'song', 'localPath', 'fileTrak', 'fileMel', 'sample/full'])


In [17]:
#Se extraen todos los datos que no requieren hacer cuentas
res = []
for genero in os.listdir(root + '/data'):
  if genero.endswith('.csv'):
    continue
  for artista in os.listdir(root + '/data/' + genero):
    for album in os.listdir(root + '/data/' + genero + '/' + artista):
      if genero == 'Instrumental':
        songName = os.path.splitext(album)[0]
        localPath = '/data/' + genero + '/' + artista
        res.append([genero, artista, genero ,songName, localPath, album])
      else:
        for song in os.listdir(root + '/data/' + genero + '/' + artista + '/' + album):
          if song.endswith((".mp3", ".wav", ".flac")):
            songName = os.path.splitext(song)[0]
            localPath = '/data/' + genero + '/' + artista + '/' + album
            res.append([genero, artista, album, songName, localPath, song])

#adding de vector to de df
df = pd.DataFrame(res, columns=['M/F/I', 'artist', 'album', 'song', 'localPath', 'fileTrak'])

In [18]:
df.to_csv(root + '/data' + '/metaData.csv', index= False)

In [ ]:
def _preacondicionar_audio(signal, native_sr):
  #resample
  if( native_sr != TARGET_SR):
    resampler = torchaudio.transforms.Resample(native_sr, TARGET_SR)
    signal = resampler(signal)
  #conversion a mono
  if(signal.shape[0] > 1):
    signal = torch.mean(signal, dim=0 ) #, keepdim= True)
  return signal, TARGET_SR
  #padding
  if(signal.shape[1] < 5*60*TARGET_SR):
    padding = torch.zeros(1, 5*60*TARGET_SR - signal.shape[1])
    signal = torch.cat((signal, padding))
  elif(signal.shape[1] > 5*60*TARGET_SR):
    signal = signal[:, :5*60*TARGET_SR]
  return signal, TARGET_SR


In [ ]:
MelCalculator = torchaudio.transforms.MelSpectrogram(sample_rate=TARGET_SR,
                                                       n_fft = N_FFT,
                                                       hop_length = HOP_LENGHT,
                                                       n_mels = N_MELS)  
  
  for i, row in df.iterrows():
    fPath = root + row['localPath'] + '/' + row['fileTrak']
    signal, sr = torchaudio.load(fPath)
    signal, sr = _preacondicionar_audio(signal, sr)
    mel = MelCalculator(signal)
    fileMel = root + row['localPath'] + '/' + row['song'] + '.pt'
    torch.save(mel, fileMel)
    #npsave(fileMel, mel)
    df.at[i, 'fileMel'] = row['song'] + '.pt'

In [ ]:
import os
songs = []
word = []
for folder in os.listdir(PATH):
  for filename in os.listdir(PATH + "/" + folder):
    if filename.endswith((".mp3", ".wav", ".flac")):
        fileDir = PATH + "/" + folder + "/" + filename
    data = [folder.split("-")[0], fileDir]
    songs.append(data)
for song in songs:
  print( f"Artista {song[0]} Cancion {song[1]}")